In [177]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import cv2
import numpy as np
import csv
from pylab import *
from matplotlib import gridspec
import datetime
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
cdict = {'red': ((0.0, 1.0, 1.0),
                 (0.125, 1.0, 1.0),
                 (0.25, 1.0, 1.0),
                 (0.5625, 1.0, 1.0),
                 (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 0.0),
                   (0.25, 0.0, 0.0),
                   (0.5625, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),
         'blue': ((0.0, 0.0, 0.0),
                  (0.5, 0.0, 0.0),
                  (1.0, 0.0, 0.0))}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)
%matplotlib inline

In [180]:
import pickle
(samples,labels) = pickle.load( open( "data_new.p", "rb" ) )


print(samples.shape)
print(labels.shape)

(73014, 10, 5, 9)
(73014,)


In [181]:
from sklearn.utils import shuffle
def class_balance(samples,labels):
    samples0 = samples[labels==0,:,:,:]
    samples1 = samples[labels==1,:,:,:]
    selected = np.random.choice(len(samples0), len(samples1), replace=False)
    samples0 = samples0[selected,:,:,:]
    labels0 = np.zeros((len(samples0)))
    labels1 = np.ones((len(samples1)))
    samples = np.concatenate((samples0,samples1),axis=0)
    labels = np.concatenate((labels0,labels1),axis=0)
    samples, labels = shuffle(samples, labels)
    return samples, labels
def one_hot(labels):
    one_hot_labels = np.zeros((labels.shape[0],2))
    one_hot_labels[labels==0,0]=1
    one_hot_labels[labels==1,1]=1
    return one_hot_labels


samples, labels = class_balance(samples,labels)

from sklearn.model_selection import train_test_split
samples, samples_rest, labels, labels_rest = \
train_test_split(samples, labels, test_size=0.3, random_state=0)
samples_validation, samples_test, labels_validation, labels_test = \
train_test_split(samples_rest, labels_rest, test_size=0.5, random_state=0)

one_hot_labels = one_hot(labels)
one_hot_labels_validation = one_hot(labels_validation)
one_hot_labels_test = one_hot(labels_test)

print(samples.shape)
print(one_hot_labels.shape)
print(samples_validation.shape)
print(one_hot_labels_validation.shape)
print(samples_test.shape)
print(one_hot_labels_test.shape)


(11088, 10, 5, 9)
(11088, 2)
(2376, 10, 5, 9)
(2376, 2)
(2376, 10, 5, 9)
(2376, 2)


In [184]:
def MinMax_Normalization(samples):
    samples_shape = samples.shape
    samples = np.reshape(samples,(samples_shape[0],samples_shape[1]*samples_shape[2]*samples_shape[3]))
    scaler = MinMaxScaler().fit(samples)
    samples_normalized = scaler.transform(samples)
    samples_normalized = np.reshape(samples_normalized,(samples_shape[0],samples_shape[1],samples_shape[2],samples_shape[3]))
    return samples_normalized, scaler

def transfer_scale(samples,scaler):
    samples_shape = samples.shape
    samples = np.reshape(samples,(samples_shape[0],samples_shape[1]*samples_shape[2]*samples_shape[3]))
    samples_normalized = scaler.transform(samples)
    samples_normalized = np.reshape(samples_normalized,(samples_shape[0],samples_shape[1],samples_shape[2],samples_shape[3]))
    return samples_normalized
    
samples_scaled, scaler = MinMax_Normalization(samples)
samples_scaled_validation = transfer_scale(samples_validation,scaler)
samples_scaled_test = transfer_scale(samples_test,scaler)

In [185]:
def feature_select(samples_scaled,features):
    return samples_scaled[:,:,:,features]

features = [0,1,2,7,8]
samples_scaled = feature_select(samples_scaled,features)
samples_scaled_validation = feature_select(samples_scaled_validation,features)
samples_scaled_test = feature_select(samples_scaled_test,features)
print(samples_scaled.shape)
print(samples_scaled_validation.shape)
print(samples_scaled_test.shape)

(11088, 10, 5, 5)
(2376, 10, 5, 5)
(2376, 10, 5, 5)


In [186]:
from keras.models import Sequential
from keras.layers import Reshape, Dense, Convolution2D, Deconvolution2D, Flatten, Input, Dropout, MaxPooling2D, Activation
from keras.models import model_from_json
from keras.activations import relu, softmax, linear
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU

In [187]:
# deep CNN model

model = Sequential()
# 10*5*5
model.add(Convolution2D(16, 3, 3,input_shape=(10, 5, 5),border_mode='valid'))
model.add(ELU())
# 8*3*16
model.add(Convolution2D(32, 3, 3,border_mode='valid'))
model.add(ELU())
# 6*1*32
model.add(Flatten())
# 192
model.add(Dense(64))
model.add(ELU())
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history_object = model.fit(samples_scaled, one_hot_labels, \
                           validation_data=(samples_scaled_validation, one_hot_labels_validation), \
                           nb_epoch=100, batch_size=64, verbose=2)

Train on 11088 samples, validate on 2376 samples
Epoch 1/100
11088/11088 [==============================] - 1s - loss: 0.6198 - acc: 0.6540 - val_loss: 0.5875 - val_acc: 0.7088
Epoch 2/100
11088/11088 [==============================] - 0s - loss: 0.5850 - acc: 0.6903 - val_loss: 0.5881 - val_acc: 0.6827
Epoch 3/100
11088/11088 [==============================] - 0s - loss: 0.5764 - acc: 0.6953 - val_loss: 0.5749 - val_acc: 0.6970
Epoch 4/100
11088/11088 [==============================] - 0s - loss: 0.5716 - acc: 0.6953 - val_loss: 0.6075 - val_acc: 0.6684
Epoch 5/100
11088/11088 [==============================] - 0s - loss: 0.5703 - acc: 0.6985 - val_loss: 0.5669 - val_acc: 0.7247
Epoch 6/100
11088/11088 [==============================] - 0s - loss: 0.5669 - acc: 0.7033 - val_loss: 0.5673 - val_acc: 0.7045
Epoch 7/100
11088/11088 [==============================] - 0s - loss: 0.5657 - acc: 0.7049 - val_loss: 0.5637 - val_acc: 0.6978
Epoch 8/100
11088/11088 [==============================

In [141]:
# deep ANN model

model = Sequential()
model.add(Flatten(input_shape=(10, 5, 5)))
# 192
model.add(Dense(64))
model.add(ELU())
model.add(Dense(32))
model.add(ELU())
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history_object = model.fit(samples_scaled, one_hot_labels, nb_epoch=10, batch_size=64, verbose=1)

Epoch 1/10
15840/15840 [==============================] - 1s - loss: 0.6122 - acc: 0.6653     
Epoch 2/10
15840/15840 [==============================] - 0s - loss: 0.5772 - acc: 0.6956     
Epoch 3/10
15840/15840 [==============================] - 0s - loss: 0.5679 - acc: 0.7033     
Epoch 4/10
15840/15840 [==============================] - 0s - loss: 0.5653 - acc: 0.7029     
Epoch 5/10
15840/15840 [==============================] - 0s - loss: 0.5603 - acc: 0.7091     
Epoch 6/10
15840/15840 [==============================] - 0s - loss: 0.5590 - acc: 0.7107     
Epoch 7/10
15840/15840 [==============================] - 0s - loss: 0.5551 - acc: 0.7119     
Epoch 8/10
15840/15840 [==============================] - 0s - loss: 0.5514 - acc: 0.7159     
Epoch 9/10
15840/15840 [==============================] - 0s - loss: 0.5489 - acc: 0.7145     
Epoch 10/10
15840/15840 [==============================] - 0s - loss: 0.5492 - acc: 0.7128     


In [55]:
model.predict(samples_scaled[0:10,:,:,:])

array([[ 0.89720249,  0.10279749],
       [ 0.9130066 ,  0.08699342],
       [ 0.90403706,  0.09596292],
       [ 0.93120247,  0.06879755],
       [ 0.85641062,  0.14358936],
       [ 0.88387764,  0.1161223 ],
       [ 0.91124195,  0.08875811],
       [ 0.95300812,  0.04699193],
       [ 0.86156821,  0.13843176],
       [ 0.89209193,  0.10790803]], dtype=float32)

In [188]:
def data_for_shallow(samples_scaled):
    num_features = samples_scaled.shape[3]
    pave_condition1 = samples_scaled[:,:,:,num_features-2]
    pave_condition1 = np.reshape(pave_condition1,(pave_condition1.shape[0],pave_condition1.shape[1]*pave_condition1.shape[2]))
    pave_condition1 = np.sum(pave_condition1,axis = 1)
    pave_condition1[pave_condition1>0]=1
    pave_condition1 = np.reshape(pave_condition1,(pave_condition1.shape[0],1))
    pave_condition2 = samples_scaled[:,:,:,num_features-1]
    pave_condition2 = np.reshape(pave_condition2,(pave_condition2.shape[0],pave_condition2.shape[1]*pave_condition2.shape[2]))
    pave_condition2 = np.sum(pave_condition2,axis = 1)
    pave_condition2[pave_condition2>0]=1
    pave_condition2 = np.reshape(pave_condition2,(pave_condition2.shape[0],1))
    
    traffic = samples_scaled[:,:,:,:num_features-2]
    data = np.reshape(traffic,(traffic.shape[0],traffic.shape[1]*traffic.shape[2]*traffic.shape[3]))
    data = np.concatenate((data,pave_condition1,pave_condition2),axis=1)
    return data

data_shallow = data_for_shallow(samples_scaled)
data_shallow_validation = data_for_shallow(samples_scaled_validation)
data_shallow_test = data_for_shallow(samples_scaled_test)

In [169]:
data_shallow.shape

(15840, 152)

In [189]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(data_shallow, labels)
pred = clf.predict(data_shallow)
acc = accuracy_score(pred, labels)
pred_validation = clf.predict(data_shallow_validation)
acc_validation = accuracy_score(pred_validation, labels_validation)
pred_test = clf.predict(data_shallow_test)
acc_test = accuracy_score(pred_test, labels_test)
print(acc)
print(acc_validation)
print(acc_test)

0.695977633478
0.693181818182
0.672138047138


In [192]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(min_samples_split=10)
clf.fit(data_shallow, labels)
pred = clf.predict(data_shallow)
acc = accuracy_score(pred, labels)
pred_validation = clf.predict(data_shallow_validation)
acc_validation = accuracy_score(pred_validation, labels_validation)
pred_test = clf.predict(data_shallow_test)
acc_test = accuracy_score(pred_test, labels_test)
print(acc)
print(acc_validation)
print(acc_test)

0.97123015873
0.765151515152
0.741161616162
